<a href="https://colab.research.google.com/github/TalHarel1998/RL_Project/blob/main/Copy_of_OpenAi_Gym_Atari.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# install dependancies, takes around 45 seconds

Rendering Dependancies



In [ ]:
#add " > /dev/null 2>&1" at the end of each command to ignore output
!pip install gym pyvirtualdisplay
!apt-get install -y xvfb python-opengl ffmpeg

Dependancies

In [ ]:
!apt-get update
!pip install --upgrade setuptools
!pip install "gym[atari]"==0.9.5
!pip install opencv-python
!pip install torch
!pip install matplotlib

In [ ]:
# ERRORS CAN BE IGNORED! all the dependency conflicts are "won" by gym!
# (maybe because of the --force parameter ?)

!pip install --force "gym[atari]"==0.9.5

In [ ]:
# download, extract & import all available ROMs to gym 
!curl http://www.atarimania.com/roms/Roms.rar -o Roms.rar
!unrar x Roms.rar
!python -m atari_py.import_roms ./

In [ ]:
%cd /content
!rm -rf RL_Project
!git clone https://github.com/TalHarel1998/RL_Project.git
%cd /content/RL_Project/dqn

# Pong

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!rm -r tmp
!rm old_statistics.pkl
!rm old_stats.pkl
!mv statistics.pkl old_statistics.pkl
!mv stats.pkl old_stats.pkl
!python main.py

In [ ]:
# !python main.py 0.6
# !rm -r tmp
# !mv statistics.pkl statistics_06.pkl
# !cp statistics_06.pkl /content/drive/MyDrive/statistics_06.pkl

# !python main.py 0.8
# !rm -r tmp
# !mv statistics.pkl statistics_08.pkl
# !cp statistics_08.pkl /content/drive/MyDrive/statistics_08.pkl

# !python main.py 0.9
# !rm -r tmp
# !mv statistics.pkl statistics_09.pkl
# !cp statistics_09.pkl /content/drive/MyDrive/statistics_09.pkl

!python main.py 0.95
!rm -r tmp
!mv statistics.pkl statistics_095.pkl
!cp statistics_095.pkl /content/drive/MyDrive/statistics_095.pkl